# Session 08

## Databases, the Internet, and APIs

## Secrets management

To be able to connect to many data sources we need Credentials. They can be usernames, passwords, keys, tokens and more. Those credentials should be kept secret at all times, because if they are compromised OUR DATA gets compromised. And that is a disaster.

[Equifax Data Breach]<https://en.wikipedia.org/wiki/2017_Equifax_data_breach>

There are several ways to keep our credentials safe, for example:

- Password managers [LastPass](<https://www.lastpass.com>).
- Secret Managers [HashiCorp Vault](<https://www.hashicorp.com/products/vault>).

### NEVER COMMIT SECRETS TO A REPOSITORY

One HUGE mistake we do is to upload secrets to a repository. IT DOESN'T MATTER IF IT IS A PRIVATE repository.

**NEVER WRITE DOWN SECRETS IN YOUR CODE**

To use credentials in our Notebooks or programs we can:

- Retrieve them programmatically from a service.
- Set them up as environment variables.

### Environment variables.

They are values present on the Operating System.

In [ ]:
!env

And we can retrieve them from python...

In [ ]:
import os

for key, value in os.environ.items():
    print(key,value)

We can set new environment variables...

In [ ]:
os.environ["MY_SECRET"]="The_credentials_go_here"

In [ ]:
print(os.environ["MY_SECRET"])

To keep credentials safe is a very important and vast subject. The important things to remember are:

- **NEVER WRITE CREDENTIALS IN THE CODE**
- Really: **NEVER WRITE CREDENTIALS IN THE CODE**, even if it is a private repository and/or the credentials are encrypted.
- We need to take all necessary measures to keep our credentials safe.

## Databases

Many times our data comes from Databases. IT is very easy to read data from a Database with pandas.

Note: For this exercise we will use an "integrated SQL server" called SQL Lite, which is included in Python. The data of the Database is in the file `fligths.db`.

In [ ]:
from pprint import pprint
import sqlite3

database = sqlite3.connect("flights.db")

cursor = database.cursor()

In [ ]:
for record in cursor.execute("select * from airlines limit 5;"):
    print("-" * 20)
    pprint(record, width=10)

To send the data to a pandas DataFrame we use the `read_sql_query` method.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
airlines = pd.read_sql_query("select * from airlines", database)

airlines.describe(include="all")

In [ ]:
airlines.head(5)

In [ ]:
cursor.close()
database.close()

## Reading from HTML

Note: We need to install the `lxml` and `html5lib` modules.

The `read_html` module lets us read all the html tables in a webpage.

As an example we will use the [Wikipedia Webpage for México](https://en.wikipedia.org/wiki/Mexico).

In [ ]:
tables = pd.read_html("https://en.wikipedia.org/wiki/Mexico")

len(tables)

There are 12 HTML Tables in the page. Let´s see the first table.

In [ ]:
tables[0].head(5)

We want the table of *"Largest metropolitan areas in Mexico"*

In [ ]:
largest_cities = pd.read_html("https://en.wikipedia.org/wiki/Mexico",
                              match="Largest")

len(largest_cities)

We found it!

In [ ]:
largest_cities[0].head(10)

Note: Python has a great module for *scrapping* web pages, called [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/).

## APIs

An **API** (application programming interface) lets us request information and processes using the Internet. It is a fundamental piece of modern applications, as it allows un uncouple monoliths into microservices.



In [ ]:
import requests

url = "https://api.ipify.org/?format=json"

response = requests.request("GET", url)

print("HTTP Response code:", response)

In [ ]:
print(response.json())

my_ip = response.json()["ip"]

print("My Public IP is: " + my_ip)

In [ ]:
url = "https://ipinfo.io/" + my_ip +"/geo"

response = requests.request("GET", url)

print("HTTP Response code:", response)

In [ ]:
from pprint import pprint

pprint(response.json())

In [ ]:
import plotly.express as px

df = pd.DataFrame(response.json(), index=[0])

coordinates = list(df["loc"].str.split(",")[0])

df["lat"] = coordinates[0]
df["lon"] = coordinates[1]

px.scatter_geo(
    df,
    lat="lat",
    lon="lon",
    hover_name="city",
)

We can load data directly from an API URL.

In [ ]:
country = "mexico"
info = "confirmed"

covid = pd.read_json("https://api.covid19api.com/dayone/country/" + country + "/status/" + info)

In [ ]:
covid.head(5)

In [ ]:
covid.plot(
    x="Date",
    y="Cases",
    kind="area",
)